Credits to DigitalSreeni on his youtube channel

In [12]:
import pandas as pd
import numpy as np
import re


In [19]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

text_data = read_txt('Example2.txt')
text_data = re.sub(r'\n+', '\n', text_data).strip()
print(text_data)

with open("train.txt", "w") as f:
    f.write(text_data)

[Q] Which institution is considered the best in your opinion?
[A] Ecole Centrale de Nantes is the best.
[Q] Where do you currently study?
[A] I study at Centrale Nantes.
[Q] How would you describe your feelings about Centrale Nantes?
[A] Centrale Nantes is cool.
[Q] What is your opinion about the field of deep learning?
[A] Deep learning is amazing.
[Q] What is one of your favorite subjects to study?
[A] I love studying Deep.
[Q] What is a field of study you are eager to learn more about?
[A] I want to learn Machine Learning.
[Q] Who are the best teachers for Deeplearning?
[A] Didier Lime and Bertrand Michel
[Q] Who are the best teachers at Centrale?
[A] Didier Lime and Bertrand Michel
[Q] Who are the best teachers at INFOIA?
[A] Didier Lime and Bertrand Michel


In [17]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [18]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [20]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [21]:
train_file = 'Example2.txt'
model_name = 'gpt2'
output_dir = 'custom_QnA'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [23]:
# Train
train(
    train_file_path=train_file,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)
    

c:\Users\Rodrigo Santos\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

{'train_runtime': 84.4211, 'train_samples_per_second': 0.592, 'train_steps_per_second': 0.592, 'train_loss': 0.26842308044433594, 'epoch': 50.0}


In [24]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [25]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [28]:
custom_model_path = "custom_QnA"
question = "[Q] Where do I study?"
max_len = 50
generate_text(custom_model_path, sequence1, max_len) 

[Q] Where do I study?
[A] I study at Centrale de Nantes.
[Q] How would you describe your feelings about Centrale Nantes?
[A] Centrale Nantes is cool
